In [1]:
import pandas as pd
import requests
import warnings
import plotly.graph_objects as go
warnings.filterwarnings('ignore')


In [2]:
# Fetching data from API
url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo"
response = requests.get(url)
data = response.json()


In [3]:
# Creating DataFrame from the fetched data
df = pd.DataFrame(data['Time Series (5min)']).T

# Converting the data columns to float
df = df.astype(float)

# Converting index to datetime
df.index = pd.to_datetime(df.index)

# Reversing the order of the DataFrame
df = df.iloc[::-1]


In [4]:
df

,1. open,2. high,3. low,4. close,5. volume
2023-09-01 10:20:00,147.475,147.53,147.395,147.430,30584.0
2023-09-01 10:25:00,147.405,147.49,147.360,147.490,18051.0
2023-09-01 10:30:00,147.500,147.53,147.405,147.431,33901.0
2023-09-01 10:35:00,147.410,147.53,147.390,147.430,22185.0
2023-09-01 10:40:00,147.415,147.57,147.380,147.500,26818.0
...,...,...,...,...,...
2023-09-01 19:25:00,147.760,147.76,147.760,147.760,6.0
2023-09-01 19:30:00,147.930,147.93,147.930,147.930,1.0
2023-09-01 19:35:00,147.930,147.93,147.930,147.930,1.0
2023-09-01 19:50:00,147.930,147.93,147.930,147.930,40.0


In [5]:
# ATR calculation
df['H-L'] = df['2. high'] - df['3. low']
df['H-PC'] = abs(df['2. high'] - df['4. close'].shift(1))
df['L-PC'] = abs(df['3. low'] - df['4. close'].shift(1))
df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
df['ATR'] = df['TR'].rolling(7).mean()

In [6]:
# Super trend indicator calculation
df['upperband'] = ((df['2. high'] + df['3. low']) / 2) + (3 * df['ATR'])
df['lowerband'] = ((df['2. high'] + df['3. low']) / 2) - (3 * df['ATR'])

df['trend'] = 0
df['trend'][0] = -1
df['supertrend'] = None

In [7]:
for i in range(0, 7):
    df['trend'][i] = -1
    df['supertrend'][i] = df['upperband'][i]

for i in range(7, len(df)):
    if df['upperband'][i] < df['upperband'][i-1] or df['4. close'][i-1] > df['upperband'][i-1]:
        df['upperband'][i] = df['upperband'][i]
    else:
        df['upperband'][i] = df['upperband'][i-1]

    if df['lowerband'][i] > df['lowerband'][i-1] or df['4. close'][i-1] < df['lowerband'][i-1]:
        df['lowerband'][i] = df['lowerband'][i]
    else:
        df['lowerband'][i] = df['lowerband'][i-1]

    if df['supertrend'][i-1] == df['upperband'][i-1]:
        if df['4. close'][i] > df['upperband'][i]:
            df['trend'][i] = 1
        else:
            df['trend'][i] = -1
    else:
        if df['4. close'][i] < df['lowerband'][i]:
            df['trend'][i] = -1
        else:
            df['trend'][i] = 1

    if df['trend'][i] == 1:
        df['supertrend'][i] = df['lowerband'][i]
    else:
        df['supertrend'][i] = df['upperband'][i]


In [12]:
df

,1. open,2. high,3. low,4. close,5. volume,H-L,H-PC,L-PC,TR,ATR,upperband,lowerband,trend,supertrend
2023-09-01 10:20:00,147.475,147.53,147.395,147.430,30584.0,0.135,NaN,NaN,0.135,NaN,NaN,NaN,-1,NaN
2023-09-01 10:25:00,147.405,147.49,147.360,147.490,18051.0,0.130,0.060,0.070,0.130,NaN,NaN,NaN,-1,NaN
2023-09-01 10:30:00,147.500,147.53,147.405,147.431,33901.0,0.125,0.040,0.085,0.125,NaN,NaN,NaN,-1,NaN
2023-09-01 10:35:00,147.410,147.53,147.390,147.430,22185.0,0.140,0.099,0.041,0.140,NaN,NaN,NaN,-1,NaN
2023-09-01 10:40:00,147.415,147.57,147.380,147.500,26818.0,0.190,0.140,0.050,0.190,NaN,NaN,NaN,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01 19:25:00,147.760,147.76,147.760,147.760,6.0,0.000,0.170,0.170,0.170,0.078571,147.94,147.758571,-1,147.94
2023-09-01 19:30:00,147.930,147.93,147.930,147.930,1.0,0.000,0.170,0.170,0.170,0.100000,147.94,147.758571,-1,147.94
2023-09-01 19:35:00,147.930,147.93,147.930,147.930,1.0,0.000,0.000,0.000,0.000,0.098571,147.94,147.758571,-1,147.94
2023-09-01 19:50:00,147.930,147.93,147.930,147.930,40.0,0.000,0.000,0.000,0.000,0.095714,147.94,147.758571,-1,147.94


In [9]:
df['trend'].value_counts()

-1    50
 1    50
Name: trend, dtype: int64

In [10]:
def create_candlestick_chart(df):
    candlestick_trace = go.Candlestick(x=df.index, open=df['1. open'], high=df['2. high'], low=df['3. low'], close=df['4. close'])
    line1_trace = go.Scatter(x=df.index, y=df['upperband'], mode='lines', name='UpperBand')
    line2_trace = go.Scatter(x=df.index, y=df['lowerband'], mode='lines', name='LowerBand')

    layout = go.Layout(
        title='Stock Price',
        xaxis_title='Time',
        yaxis_title='Stock Price'
    )

    fig = go.Figure(data=[candlestick_trace, line1_trace, line2_trace], layout=layout)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()


In [11]:
create_candlestick_chart(df)

In this code, we fetched data from an API, created a DataFrame from the fetched data, and calculated the Super Trend indicator using the fetched data. The Super Trend indicator helps identify the direction of the current trend in the market. By analyzing the upper and lower bands of the Super Trend indicator, traders can make informed decisions about buying or selling assets.

---

The code begins by fetching the data, converting it into a DataFrame, and performing data preprocessing tasks like converting columns to float and indexing by datetime. It then calculates the ATR and Super Trend indicators based on specified parameters. Finally, it generates a candlestick chart using Plotly to visualize the market data and Super Trend bands.